## 모듈 Import

In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from glob import glob
from scipy import interpolate
import warnings
import datetime
import seaborn as sns
warnings.filterwarnings("ignore")
sys.path.append(str(Path(os.getcwd())))

## Path 설정

In [2]:
base_path = Path(os.getcwd()).parent
sys.path.append(base_path)

## 데이터 로딩

In [3]:
water_lst = glob(f'{base_path}/raw_data/water_data/*.csv')
rain_lst = glob(f'{base_path}/raw_data/rf_data/*.csv')

## 데이터 병합

In [4]:
water_lst = glob(f'{base_path}/raw_data/water_data/*.csv')
rain_lst = glob(f'{base_path}/raw_data/rf_data/*.csv')

water_df = pd.DataFrame()
rain_df = pd.DataFrame()
for w in water_lst:
    water_df = water_df.append(pd.read_csv(w))
for r in rain_lst:
    rain_df = rain_df.append(pd.read_csv(r))

## 데이터 명세

In [5]:
# water 데이터 컬럼 설명
"""
ymdhm : 년월일시분
swl : 팔당댐 현재수위 (단위 :El.m)
inf : 팔당댐 유입량 (단위 : m^3/s)
swf : 팔당댐 저수량 (단위 : 만m^3)
ecpc : 팔당댐 공용량 (단위 : 백만m^3)
tototf: 총 방류량 (eksdnl : m^3/s)
tide_level : 강화대교 조위 ( 단위 : cm) # 조위는 조석에 의해 변하는 해수면의 높이
wl_1018662 : 청담대교 수위 ( 단위 : cm)
fw_1018622 : 청담대교 유량 ( 단위 : m^3/s)
(유량은 단위시간 당 얼마만큼의 체적에 해당하는 액체가 이동했는지 나타낼 때 쓰는 개념)
wl_1018680 : 잠수교 수위 ( 단위 : cm)
fw_1018680 : 잠수교 유량 ( 단위 : m^3/s)
wl_1018683 : 한강대교 수위 ( 단위 : cm)
fw_1018683 : 한강대교 유량 ( 단위 : m^3/s)
wl_1019630 : 행주대교 수위 ( 단위 : cm)
fw_1019630 : 행주대교 유량 ( 단위 : m^3/s)
"""

# RainFall 데이터 컬럼 설명
"""
YMDHM : 년월일시분
rf_10184100 : 대곡교 강수량(단위 : cm)
rf_10184110 : 진관교 강수량(단위 : cm)
rf_10184140 : 송정동 강수량(단위 : cm)
"""

'\nYMDHM : 년월일시분\nrf_10184100 : 대곡교 강수량\nrf_10184110 : 진관교 강수량\nrf_10184140 : 송정동 강수량\n'

## 데이터 타입 수정

In [56]:
water_df['ymdhm'] = pd.to_datetime(water_df['ymdhm'], format = '%Y-%m-%d %H:%M:%S' )
rain_df['ymdhm'] = pd.to_datetime(water_df['ymdhm'], format = '%Y-%m-%d %H:%M:%S' )

In [57]:
merge_df = pd.merge(water_df, rain_df, how = 'left', on = 'ymdhm')

In [62]:
merge_df

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
0,2012-05-01 00:00:00,24.800,555.00,219.07,24.93,555.00,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18,0.0,0.0,0.0
1,2012-05-01 00:10:00,24.794,464.60,218.86,25.15,562.90,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18,0.0,0.0,0.0
2,2012-05-01 00:20:00,24.789,478.10,218.69,25.31,576.40,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18,0.0,0.0,0.0
3,2012-05-01 00:30:00,24.789,464.80,218.69,25.31,563.10,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17,0.0,0.0,0.0
4,2012-05-01 00:40:00,24.789,478.10,218.69,25.31,576.40,450.0,311.7,476.21,301.2,0.0,291.0,707.17,277.3,564.29,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276331,2022-07-18 23:10:00,25.040,259.23,212.86,31.14,259.23,510.0,0.0,319.84,0.0,NaN,0.0,-456.41,0.0,974.40,0.0,0.0,0.0
276332,2022-07-18 23:20:00,25.040,260.46,212.86,31.14,260.46,492.0,0.0,314.01,0.0,NaN,0.0,-717.30,0.0,1006.88,0.0,0.0,0.0
276333,2022-07-18 23:30:00,25.040,259.37,212.86,31.14,259.37,475.0,0.0,387.55,0.0,NaN,0.0,-843.37,0.0,1039.90,0.0,0.0,0.0
276334,2022-07-18 23:40:00,25.040,259.13,212.86,31.14,259.13,458.0,0.0,454.91,0.0,NaN,0.0,-1023.37,0.0,1073.46,0.0,0.0,0.0


## 데이터셋 분리(train, valid, test)

In [69]:
# test set: 2022 06 01 ~
train = merge_df[-((merge_df['ymdhm'].dt.year==2022) & (merge_df['ymdhm'].dt.month>=6))].reset_index(drop =True)
test = merge_df[((merge_df['ymdhm'].dt.year==2022) & (merge_df['ymdhm'].dt.month>=6))].reset_index(drop =True)

In [72]:
test.isna().sum()

ymdhm             0
swl              36
inf              36
sfw              36
ecpc             36
tototf           36
tide_level        4
wl_1018662        0
fw_1018662     1216
wl_1018680        0
fw_1018680     6912
wl_1018683        0
fw_1018683        0
wl_1019630        0
fw_1019630        0
rf_10184100       0
rf_10184110       0
rf_10184140       0
dtype: int64

- 테스트 데이터의 강화대교와 잠수교의 유량이 결측값으로 존재 이러한 경우 유량이 예측에 중요 변수일경우 예측정확도 감소
- 강화대교와 잠수교의 유랑을 아래 다른 두개의 대교의 유량 혹은 강화대교와 잠수교의 유량측정 이전단계 강수량 혹은 수위를 통해서 예측할 수 있는지 확인해보기